# ***Ознакомившись с описанием датасета, я считаю, что целевой (target) переменной прогнозой модели является анализ на основе имеющихся данных различных признаков Zomato Restaurants, а также, взаимосвязей этих признаков. Для чего же нам необходим этот анализ? На основе его результатов, мы сможем проследить некоторые закономерности между признаками и делать выводы о необходимости изменения тех или иных "фич", с целью повышения рейтинга ресторанов, а это, в свою очередь, необходимо для увеличения прибыли с них, что и является, по моему мнению, основной задачей прогнозной модели.*** 

 # Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузите данные. Они находятся в директории input.

In [ ]:
zomato1 = pd.read_csv('../input/zomato.csv',encoding="ISO-8859-1")
country_code = pd.read_excel('../input/Country-Code.xlsx')
zomato = pd.merge(zomato1,country_code, on='Country Code') #Объединим zomato1 и country_code в zomato, чтобы иметь код страны и её название в одном наборе данных.
zomato.head().T

# В каких странах расположены рестораны? Назовите топ-5 стран. Выведите аналогичную статистику по городам.

***Все страны, в которых расположены рестораны:***

In [ ]:
zomato['Country'].unique()

***Топ-5 стран по количеству ресторанов zomato:***

In [ ]:
zomato['Country'].value_counts().head()

***Все города, в которых расположены рестораны:***

In [ ]:
zomato['City'].unique()

***Топ-5 городов по количеству ресторанов zomato:***

In [ ]:
zomato['City'].value_counts().head()

***График по топ-5 странам:***

In [ ]:
zomato['Country'].value_counts().head().plot(kind='bar', title='Number of zomato restaurants in top-5 countries')

***График по топ-5 городам:***

In [ ]:
zomato['City'].value_counts().head().plot(kind='bar',title='Number of zomato restaurants in top-5 cities')

# **Какие типы кухни подают в ресторанах? Как связан этот признак с рейтингом? Не забывайте о визуализации информации.**

***Типы кухни, которые подают в ресторанах:***

In [ ]:
def func(s):
    return s.split(', ')

cuisines_list1 = zomato['Cuisines'].unique()
cuisines_list2 = []
for i in range (cuisines_list1.size):
    cuisines_list2 += func(str(cuisines_list1[i]))
cuisines_list = list(set(cuisines_list2))
cuisines_list 

***Средний рейтинг ресторанов в зависимости от типов кухни:***

In [ ]:
zomato.groupby('Cuisines')['Aggregate rating'].mean().sort_values(ascending=False)

***График топ-50 типов кухни:***

In [ ]:
zomato.groupby('Cuisines')['Aggregate rating'].mean().sort_values(ascending=False).head(50).plot(kind='bar',figsize=(20,7),title='Aggregate rating top-50 cuisines')

# Что можно сказать о связи ценового диапазона и рейтинга ресторана? Не забывайте о визуализации информации.

***связь ценового диапазона и рейтинга ресторана:***

Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) между ценовым диапазоном и рейтингом ресторана ***статистически значима***.

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = spearmanr(zomato['Price range'], zomato['Aggregate rating'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

***Посмотрим какой будет средний рейтинг ресторанов в зависимости от их ценового диапазона:***

In [ ]:
zomato.groupby('Price range')['Aggregate rating'].mean().sort_values(ascending=False)

***Построим график этой зависимости:***

In [ ]:
zomato.groupby('Price range')['Aggregate rating'].mean().sort_values(ascending=False).plot(kind='bar',title='Relation between Price range and Aggregate rating')

# Есть ли различия в рейтинге ресторанов с возможностью бронирования столиков и без него? 

***Проверим зависит ли рейтинг ресторана от возможности бронирования столиков.***

***Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) между рейтингом ресторана и возможностью бронирования столиков в нём - статистически значима, а это значит, что различия есть.***

In [ ]:
new_values = {'Yes':1, 'No':0}
zomato['Dummy Has Table booking'] = zomato['Has Table booking'].map(new_values)

from scipy.stats import pointbiserialr
pointbiserialr(zomato['Aggregate rating'], zomato['Dummy Has Table booking'])

***Посмотрим какой будет средний рейтинг ресторанов в зависимости от возможности бронирования столиков:***

In [ ]:
zomato.groupby('Has Table booking')['Aggregate rating'].mean().sort_values(ascending=False)

***Построим график для наглядности:***

In [ ]:
zomato.groupby('Has Table booking')['Aggregate rating'].mean().sort_values(ascending=False).plot(kind='bar',title='Relation between Booking and Aggregate rating')

# Визуализируйте признак **Rating color** (или **Rating text**) в комбинации с геолокацией (**Longitude**, **Latitude**). Какой вывод можно сделать?

In [ ]:
colorscale = [
[0, 'rgb(255, 255, 255)'], 
[0.1, 'rgb(255, 255, 255)'],
[0.2, 'rgb(212, 28, 28)'], 
[0.3, 'rgb(212, 28, 28)'],
[0.4, 'rgb(255, 186, 59)'], 
[0.5, 'rgb(255, 186, 59)'],
[0.6, 'rgb(248, 255, 59)'], 
[0.7, 'rgb(248, 255, 59)'], 
[0.8, 'rgb(0, 227, 30)'], 
[0.9, 'rgb(0, 227, 30)'],
[1, 'rgb(0, 117, 16)']
]

fig1 = [dict(
    type='scattergeo',
    lon = zomato['Longitude'],
    lat = zomato['Latitude'],
    text = zomato['Restaurant Name'],
    mode = 'markers',
    marker = dict(
    color = zomato['Rating color'],
    cmin = zomato['Aggregate rating'].min(),
    cmax = zomato['Aggregate rating'].max(),
    colorscale=colorscale,
    colorbar = dict(
                title = 'Rating'
            )
        )
    )]

fig_layout = dict(
    title = 'Rating of zomato restaurants on the world map'
)
fig = go.Figure(data=fig1, layout=fig_layout)
iplot(fig)

***По полученной карте можно сделать вывод, что самый высокий рейтинг имеют рестораны расположенные в Индии, Европе, а также на юго-востоке США. Как видно, больше всего ресторанов Zomato находится в Индии.***

# Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

**--------------------------------------------------------------------------------------------------------------------------------------------------**

# Давайте по имеющимся данным найдем топ-20 ресторанов с наибольшим количеством отзывов.

***Топ-20 ресторанов с наибольшим количеством отзывов:***

In [ ]:
max_votes = zomato['Votes'].sort_values(ascending=False).head(20)
zomato.loc[zomato['Votes'].isin(max_votes)][['Restaurant Name','Votes']]

***Построим график:***

In [ ]:
max_votes = zomato['Votes'].sort_values(ascending=False).head(20)
zomato.loc[zomato['Votes'].isin(max_votes)][['Restaurant Name','Votes']].plot(kind='bar', x = 'Restaurant Name', y = 'Votes', figsize=(20,10), color = 'darkblue')

# Давайте узнаем есть ли связь между средней ценой за двоих и общим рейтингом ресторана. (Будем проводить анализ по Индии, ибо в разных странах разные валюты)

***Проверим есть ли связь:***

In [ ]:
zomato_india = zomato.loc[zomato['Country']=='India']

from scipy.stats import pearsonr, spearmanr, kendalltau
r = spearmanr(zomato_india['Average Cost for two'], zomato_india['Aggregate rating'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

***Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) статистически значима.***

***Посмотрим как средний рейтинг ресторана зависит от средней цены за двоих:***

In [ ]:
zomato_india.plot.scatter(x='Average Cost for two',y='Aggregate rating',figsize=(15,7), title="Cost vs Agg Rating")

# Ранее мы смотрели как связан средний рейтинг ресторана с типом кухни, теперь давайте посмотрим какие типы кухни наиболее популярны.

***Типы кухни отсортированные по популярности:***

In [ ]:
zomato['Cuisines'].value_counts()

***Построим график топ-20 типов кухни:***

In [ ]:
zomato['Cuisines'].value_counts().head(20).plot(kind='bar', figsize = (20,7))

# Теперь давайте проверим какие рестораны в нашем наборе данных имеют наибольшее число точек.

In [ ]:
zomato['Restaurant Name'].value_counts()

***Построим график топ-20 ресторанов по количеству точек:***

In [ ]:
zomato['Restaurant Name'].value_counts().head(20).plot(kind='pie',figsize=(15,15), title="Top 15 Restaurants with maximum outlets", autopct='%1.2f%%')

# Давайте сделаем анализ корреляций числовых признаков.

Выделим числовые признаки в отдельную группу.

In [ ]:
numeric = ['Price range', 'Aggregate rating', 'Votes', 'Longitude', 'Latitude']
sns.pairplot(zomato[numeric])

***Удалим выбросы.***

In [ ]:
sns.boxplot(zomato['Price range']);

In [ ]:
sns.boxplot(zomato['Aggregate rating']);

In [ ]:
sns.boxplot(zomato['Votes']);

In [ ]:
sns.boxplot(zomato['Longitude']);

In [ ]:
sns.boxplot(zomato['Latitude']);

***Для удаления выбросов воспользуемся правилом трёх сигм:***

In [ ]:
def outliers_indices(feature):
    mid = zomato[feature].mean()
    sigma = zomato[feature].std()
    return zomato[(zomato[feature] < mid - 3*sigma) | (zomato[feature] > mid + 3*sigma)].index

wrong_price_range = outliers_indices('Price range')
wrong_aggregate_rating = outliers_indices('Aggregate rating')
wrong_votes = outliers_indices('Votes')
wrong_longitude = outliers_indices('Longitude')
wrong_latitude = outliers_indices('Latitude')

out = set(wrong_price_range) | set(wrong_aggregate_rating) | set(wrong_votes) | set(wrong_longitude) | set(wrong_latitude)
print(len(out))

***Из набора данных будет удалено 779 объектов-выбросов, что не является существенным в данном случае.***

In [ ]:
zomato.drop(out, inplace=True)

***Построим матрицу корреляций Спирмена для числовых признаков без учёта объектов-выбросов.***

In [ ]:
zomato[numeric].corr(method='spearman')

***Воспользуемся функцией heatmap для визуализации:***

In [ ]:
sns.heatmap(zomato[numeric].corr(method='spearman'));